# Aprendizaje Supervisado
# UdeSA, 2023
## Profesores: Marcela Svarc, Juan Manuel Pérez


## TP: Clasificación de tópicos de noticias



In [1]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 10.5 MB/s eta 0:00:00


Vamos a comenzar viendo de armar un modelo de bolsa de palabras. Para ello, utilizaremos `CountVectorizer` de `sklearn`


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

CountVectorizer?

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

textos = [
    "bolsa de palabras",
    "palabras de bolsa",
    "bolsa es una palabra",
    "palabra es una bolsa",
    "palabra no es una bolsa",
    "bolsa es una bolsa",
    "bolsa es una bolsa y es una palabra",
    "bolsa bolsa bolsa palabra es es una una y",
]

vect = CountVectorizer()

vect.fit(textos)

CountVectorizer()

En `vect.vocabulary_` podemos encontrar el vocabulario construido por `CountVectorizer`

In [4]:
vect.vocabulary_

{'bolsa': 0, 'de': 1, 'palabras': 5, 'es': 2, 'una': 6, 'palabra': 4, 'no': 3}

`vect.transform` convierte un conjunto de textos en su representación matricial

In [6]:
vect.transform(textos).todense()

matrix([[1, 1, 0, 0, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 0],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 0, 1, 1, 1, 0, 1],
        [2, 0, 1, 0, 0, 0, 1],
        [2, 0, 2, 0, 1, 0, 2],
        [3, 0, 2, 0, 1, 0, 2]])

Armemos un dataframe para que nos muestre qué significa cada columna de la matriz

In [7]:
import pandas as pd

# Doy vuelta y ordeno el vocabulario (perdón por lo horrendo)
vocab = {v:k for k, v in vect.vocabulary_.items()}
columns = [vocab[k] for k in range(len(vocab))]

df = pd.DataFrame(
    vect.transform(textos).todense(),
    columns=columns
)

df["text"] = textos

df

,bolsa,de,es,no,palabra,palabras,una,text
0,1,1,0,0,0,1,0,bolsa de palabras
1,1,1,0,0,0,1,0,palabras de bolsa
2,1,0,1,0,1,0,1,bolsa es una palabra
3,1,0,1,0,1,0,1,palabra es una bolsa
4,1,0,1,1,1,0,1,palabra no es una bolsa
5,2,0,1,0,0,0,1,bolsa es una bolsa
6,2,0,2,0,1,0,2,bolsa es una bolsa y es una palabra
7,3,0,2,0,1,0,2,bolsa bolsa bolsa palabra es es una una y


Este modelo se llama de "unigramas" porque sólo considera apariciones de una única palabra. ¿Podemos extender esto?

## n-gramas

Una forma de agregar cierto orden  es mediante el uso de n-gramas.

In [11]:

vect = CountVectorizer(ngram_range=(1, 2))

vect.fit(textos)

# Doy vuelta y ordeno el vocabulario (perdón por lo horrendo)
vocab = {v:k for k, v in vect.vocabulary_.items()}
columns = [vocab[k] for k in range(len(vocab))]

df = pd.DataFrame(
    vect.transform(textos).todense(),
    columns=columns
)

df["text"] = textos


pd.options.display.max_columns = 50

df

,bolsa,bolsa bolsa,bolsa de,bolsa es,bolsa palabra,de,de bolsa,de palabras,es,es es,es una,no,no es,palabra,palabra es,palabra no,palabras,palabras de,una,una bolsa,una palabra,una una,text
0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,bolsa de palabras
1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,palabras de bolsa
2,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,bolsa es una palabra
3,1,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1,0,0,palabra es una bolsa
4,1,0,0,0,0,0,0,0,1,0,1,1,1,1,0,1,0,0,1,1,0,0,palabra no es una bolsa
5,2,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,bolsa es una bolsa
6,2,0,0,2,0,0,0,0,2,0,2,0,0,1,0,0,0,0,2,1,1,0,bolsa es una bolsa y es una palabra
7,3,2,0,0,1,0,0,0,2,1,1,0,0,1,1,0,0,0,2,0,0,1,bolsa bolsa bolsa palabra es es una una y


## Conjunto de datos



Vamos a utilizar el conjunto de datos [AG News](https://huggingface.co/datasets/ag_news). Cada instancia del conjunto de datos consiste en un título y un cuerpo de una noticia, y una etiqueta que indica la categoría a la que pertenece. Las categorías son: 'World', 'Sports', 'Business' y 'Sci/Tech'.

| Texto                                                                                           | Categoría |
| ----------------------------------------------------------------------------------------------- | --------- |
| Wall St. Bears Claw Back Into the Black (Reuters) Reuters -.                                    | Business  |
| VIEW: Arafats ghost Barry Rubin Rarely in history has a political movement ...                  | World     |
| Tough loss for Pavano, Marlins ATLANTA -- All-Star pitcher Carl Pavano ...                      | Sports    |
| Digital ways to have fun including an HP-branded version of Apple's iPod - designed with an ... | Sci/Tech  |


In [12]:
from datasets import load_dataset

# Con esto lo bajamos de este repositorio centralizado
ds = load_dataset("ag_news")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [13]:
# Convertimos a pandas

id2label = ds["train"].features["label"].names

df_train = ds["train"].to_pandas()

df_test = ds["test"].to_pandas()

In [14]:
df_train

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
119995,Pakistan's Musharraf Says Won't Quit as Army C...,0
119996,Renteria signing a top-shelf deal Red Sox gene...,1
119997,Saban not going to Dolphins yet The Miami Dolp...,1
119998,Today's NFL games PITTSBURGH at NY GIANTS Time...,1


In [15]:
id2label

['World', 'Sports', 'Business', 'Sci/Tech']

Tips:

- Usar `Pipeline` de `sklearn` para armar el pipeline de preprocesamiento y clasificación todo en uno. Mirar la documentación para ver cómo se usa.

- Pegarle un vistazo a Tf-Idf en la parte de "Feature Extraction" de `sklearn`.